# Sentinel-2 satellite image analysis

This notebook intends to show couple ideas how to analyse satellite images using different spectral bands and visualising the output.

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from cscale_notebooks_functions import load_sentinel_image, display_rgb, image_rgb, normalized_difference, plot_masked_rgb, resampling

In [ ]:
# Choose folder where are your downloaded satelite images  
img_folder = "/images/clipped/"
path = os.path.abspath("")
folder_img = path + img_folder

Different channels has different resolution, so it needs to be resampled in order to be able to put them together. You can check the channel resolution [here](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spatial).

In case of L1C data we need to resample channels B05, B06, B11 and B12 with the factor of 2 and B09 with factor 6.

In [ ]:
file_temp = "B02.tif"
file_res = ["B05.tif", "B06.tif", "B11.tif", "B12.tif"]

In [ ]:
for n in range(len(file_res)):
    resampled = resampling(folder_img,file_temp,file_res[n],2)

NOTE: In case of L1C data the channel B09 is resampled with factor of 6, whereas L2A data needs resampling of ONLY channel B09 with the factor of 3!

In [ ]:
file_temp = "B02.tif"
file_res = ["B09.tif", "B10.tif"]

for n in range(len(file_res)):
    resampled_09 = resampling(folder_img,file_temp,file_res[n],6)

NOTE: In case of L1C data we need to enter B05_res, B06_res channels. For L2A just B05 and B06.

In [ ]:
img_rgb = load_sentinel_image(folder_img, ["B02", "B03", "B04"])
img_nir = load_sentinel_image(folder_img, ["B03", "B04", "B06_res"])

rgb = image_rgb(img_rgb, 'B04', 'B03', 'B02', alpha=5.)
nir = image_rgb(img_nir, 'B06_res', 'B04', 'B03', alpha=5.)

We can compute spectral index using different bands. In this notebook we calculate Normalized Difference (ND) index using custom function. ND index is defined as:

$$ 
ND = (Band1 - Band2)/ (Band1 + Band2) 
$$

Here we calculate two indexes, where different areas are emphasized.

In [ ]:
img_nir2 = load_sentinel_image(folder_img, ["B03", "B04", "B08"])

# Calculating two indices
ndvi = normalized_difference(img_nir2, 'B08', 'B04')
mndwi = normalized_difference(img_nir2, 'B03', 'B08')

In [ ]:
from matplotlib import pyplot as plt
# checking the images
fig, ax = plt.subplots(2, 2, figsize=(20, 15))

fontsize=16
ax = ax.flatten()

ax[0].imshow((rgb*255).astype(np.uint8), interpolation='none')
ax[0].axis('off')
ax[0].set_title('Color Image', fontsize=fontsize)

ax[1].imshow((nir*255).astype(np.uint8), interpolation='none')
ax[1].axis('off')
ax[1].set_title('NIR', fontsize=fontsize)

ax[2].imshow(ndvi, cmap='Greens', interpolation='none')
ax[2].axis('off')
ax[2].set_title('NDVI', fontsize=fontsize)

ax[3].imshow(mndwi, cmap='Blues', interpolation='none') # * 255).astype(np.uint8)
ax[3].axis('off')
ax[3].set_title('MNDWI', fontsize=fontsize)

plt.subplots_adjust(wspace=0.1, hspace=0.1)

plt.show()

Colour image is a true colour image, what one can observe for example from a plane. NIR image is more sensitive to different kind of vegetation and water areas. NDVI index gives a better response (higher values = greener because of the colormap we applied) over the vegetation areas and the MNDWI has a higher response (higher values = bluer because of the ‘Blues’ colormap) for water areas like rivers.

We can create mask for water areas using MNDWI index. Setting threshold for this index we can display only pixels with higher value representing water area and zero value for the rest. It needs to be adjusted for different data.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive

def watermask(treshold):
    water_mask = mndwi > treshold
    
    plt.figure(figsize=(10,10))
    plt.imshow(water_mask)
    plt.axis("off")
    plt.show()
    return water_mask

water_mask = interactive(watermask, treshold = (-0.5,0.5,0.01))
display(water_mask)

In [ ]:
mask = water_mask.result
shape = img_rgb['B02'].shape
mask = mask[:shape[0], :shape[1]]

mask = np.where(img_rgb['B02'] == 0, 0, mask)

masked_rgb = plot_masked_rgb(red=img_rgb['B04'],
                             green=img_rgb['B03'],
                             blue=img_rgb['B02'],
                             mask=mask,
                             color_mask=(0, 0.6, 1),
                             transparency=0.0,
                             brightness=4
                            )

plt.figure(figsize=(15, 10))
plt.imshow((masked_rgb * 255).astype(np.uint8))
plt.axis("off")
plt.show()

We can play more with the bands and their order while displaying.

In case of L1C data we need to enter B12_res and B11_res. L2A just B12 and B11.

In [ ]:
img = load_sentinel_image(folder_img, ["B03", "B04", "B12_res"])
rgb = image_rgb(img, 'B12_res', 'B03', 'B04', alpha=5.)
bgr = image_rgb(img, 'B04', 'B12_res', 'B03', alpha=5.)

fig, ax = plt.subplots(1, 2, figsize=(20, 15))
ax[0].imshow((rgb* 255).astype(np.uint8), cmap='Greens')
ax[0].axis('off')
ax[1].imshow((bgr* 255).astype(np.uint8), cmap='Blues')
ax[1].axis('off')
plt.show()

## We can narrow down our analysis for a city region.

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.plot import show
from cscale_notebooks_functions import coor_converter

coor_converter("geojson/prague.geojson","geojson/prague_GPScut.geojson",32633)

Prague_reg = "geojson/prague_GPScut.geojson"

gdf = gpd.read_file(Prague_reg)

rgb_tci = rasterio.open(folder_img + "TCI.tif")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 7))
show(rgb_tci,ax=ax)
gdf.plot(ax=ax, color='yellow', alpha=.3, aspect=1)
plt.axis("off")
plt.show()

In [ ]:
import os
from cscale_notebooks_functions import clipper
#directory = "Prague_sunny_clipped/"
roicut_path = path + "/images/cut_out/"
if os.path.isdir(roicut_path) != True:
    os.mkdir(roicut_path)

In [ ]:
clipper(folder_img, Prague_reg, roicut_path, whole = "yes")

## We can look for parks, water surfaces or buildings in city area

In [ ]:
img = load_sentinel_image(roicut_path, ["B03", "B04", "B09_res"])
img1 = load_sentinel_image(roicut_path, ["B03", "B04", "B09_res"])
rgb = image_rgb(img, 'B03', 'B04', 'B09_res', alpha=5.5)
bgr = image_rgb(img1, 'B04', 'B09_res', 'B03', alpha=5.5)

fig, ax = plt.subplots(1, 2, figsize=(25, 20))
ax[0].imshow((rgb* 255).astype(np.uint8))
ax[0].axis('off')
ax[1].imshow((bgr* 255).astype(np.uint8))
ax[1].axis('off')
plt.show()

## Setting mask for green areas

In [ ]:
diff = normalized_difference(img,'B09_res','B03',)

plt.figure(figsize=(5,5))
plt.imshow(diff, cmap = 'Greens_r')
plt.axis("off")
plt.show()

Value of masks needs to be adjusted.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive

def greenmask(treshold):
    green_mask = diff > treshold
    
    plt.figure(figsize=(10,10))
    plt.imshow(green_mask)
    plt.axis("off")
    plt.show()
    return green_mask

gm = interactive(greenmask, treshold = (-1.0,1.0,0.01))
display(gm)

## Overlay image with highlighted green and water areas

In [ ]:
img_crgb = load_sentinel_image(roicut_path, ["B02", "B03", "B04"])
mask = gm.result
shape = img_crgb['B02'].shape
mask = mask[:shape[0], :shape[1]]

mask = np.where(img_crgb['B02'] == 0, 0, mask)
 
masked_crgb = plot_masked_rgb(red=img_crgb['B04'],
                             green=img_crgb['B03'],
                             blue=img_crgb['B02'],
                             mask=mask,
                             color_mask=(0.3, 0.7, 0),
                             transparency=0.1,
                             brightness=4
                            )

plt.figure(figsize=(10, 5))
plt.imshow((masked_crgb * 255).astype(np.uint8))
plt.axis("off")
plt.show()

Adding water_mask.

In [ ]:
img_nir2 = load_sentinel_image(roicut_path, ["B03", "B04", "B08"])

mndwi = normalized_difference(img_nir2, 'B03', 'B08')

def watermask(treshold):
    water_mask_crgb = mndwi > treshold
    
    plt.figure(figsize=(10,5))
    plt.imshow(water_mask_crgb)
    plt.axis("off")
    plt.show()
    return water_mask_crgb

wm = interactive(watermask, treshold = (-0.5,0.5,0.01))
display(wm)

In [ ]:
mask = wm.result
masked_rgb_trees = masked_crgb
shape = masked_rgb_trees[:,:,0].shape
mask = mask[:shape[0], :shape[1]]

mask = np.where(masked_rgb_trees[:,:,0] == 0, 0, mask)

masked_rgb_wt = plot_masked_rgb(masked_rgb_trees[:,:,0],masked_rgb_trees[:,:,1],masked_rgb_trees[:,:,2],
                             mask=mask,
                             color_mask=(0.1, 0.7, 1),
                             transparency=0,
                             brightness=4.
                            )

plt.figure(figsize=(10, 5))
plt.imshow((masked_rgb_wt * 255).astype(np.uint8))
plt.axis("off")
plt.show()

## Saving images into TIF file.

In [ ]:
from PIL import Image
img_overlay = Image.fromarray((masked_rgb_wt * 255).astype(np.uint8), "RGB")

#Choose image name
image_filename = "Prague_overlay.tif"

img_overlay.save(image_filename)
print(f"Image has been saved to {image_filename} file")

File can be loaded again using [matplotlib](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html).

In [ ]:
img = plt.imread(image_filename)
plt.imshow(img)